
- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [95]:
# (this is not an exhaustive list of libraries)

import pandas as pd
from pandas import json_normalize
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

In [96]:
# load one file first to see what type of data you're dealing with and what attributes it has

directory = r"C:\Users\turab\data-project-midterm\data"

# Use os.listdir to get files
files = os.listdir(directory)

# Print all files
for file in files:
    print(file)

.gitkeep
AK_Juneau_0.json
AK_Juneau_1.json
AK_Juneau_2.json
AK_Juneau_3.json
AK_Juneau_4.json
AL_Montgomery_0.json
AL_Montgomery_1.json
AL_Montgomery_2.json
AL_Montgomery_3.json
AL_Montgomery_4.json
AR_LittleRock_0.json
AR_LittleRock_1.json
AR_LittleRock_2.json
AR_LittleRock_3.json
AR_LittleRock_4.json
AZ_Phoenix_0.json
AZ_Phoenix_1.json
AZ_Phoenix_2.json
AZ_Phoenix_3.json
AZ_Phoenix_4.json
CA_Sacramento_0.json
CA_Sacramento_1.json
CA_Sacramento_2.json
CA_Sacramento_3.json
CA_Sacramento_4.json
CO_Denver_0.json
CO_Denver_1.json
CO_Denver_2.json
CO_Denver_3.json
CO_Denver_4.json
CT_Hartford_0.json
CT_Hartford_1.json
CT_Hartford_2.json
CT_Hartford_3.json
CT_Hartford_4.json
DE_Dover_0.json
DE_Dover_1.json
DE_Dover_2.json
DE_Dover_3.json
DE_Dover_4.json
FL_Tallahassee_0.json
FL_Tallahassee_1.json
FL_Tallahassee_2.json
FL_Tallahassee_3.json
FL_Tallahassee_4.json
GA_Atlanta_0.json
GA_Atlanta_1.json
GA_Atlanta_2.json
GA_Atlanta_3.json
GA_Atlanta_4.json
HI_Honolulu_0.json
HI_Honolulu_1.json
HI_

Combine all sales into dataframe

In [97]:
def load_and_combine_json_files(directory):
    data_frames = []
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                
                # Normalize the JSON structure to a flat table
                if 'data' in json_data and 'results' in json_data['data']:
                    df = json_normalize(json_data['data']['results'])
                    data_frames.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

directory = r"C:\Users\turab\data-project-midterm\data"  # Update this path

# Load and combine the JSON files
combined_data = load_and_combine_json_files(directory)

print(combined_data.head())

C:\Users\turab\AppData\Local\Temp\ipykernel_16660\2437266733.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(data_frames, ignore_index=True)


       last_update_date                                               tags  \
0  2023-09-19T20:52:50Z  [carport, community_outdoor_space, cul_de_sac,...   
1                  None                                               None   
2                  None                                               None   
3                  None                                               None   
4                  None                                               None   

                                           permalink status  \
0       9453-Herbert-Pl_Juneau_AK_99801_M90744-30767   sold   
1  8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...   sold   
2      4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516   sold   
3  17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...   sold   
4  9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...   sold   

                     list_date open_houses  \
0  2023-06-29T21:16:25.000000Z        None   
1                         None        None   
2             

At this point, ensure that you have all sales in a dataframe.
- Is each cell one value, or do some cells have lists?
- Maybe the "tags" will help create some features.
- What are the data types of each column?
- Some sales may not actually include the sale price.  These rows should be dropped.
- Some sales don't include the property type.
- There are a lot of None values.  Should these be dropped or replaced with something?

Drop or replace values as neccesary 



In [98]:
# Check if some columns have lists

list_columns_combined_data = check_for_lists(combined_data)
print(list_columns_combined_data)

# Tags, branding, photos, virtual tours, source agents, other listings, etc have lists!

NameError: name 'check_for_lists' is not defined

In [99]:
# Display all from tags column 

if 'tags' in combined_data.columns:
    # Initialize an empty set to store unique tags
    unique_tags = set()
    
    # Iterate over each entry in the 'tags' column
    for tag_list in combined_data['tags']:
        if isinstance(tag_list, list):  # Ensure the entry is a list
            unique_tags.update(tag_list)  # Add tags to the set, avoiding duplicates
    
    # Print the unique set of tags
    print("Unique tags:")
    for tag in unique_tags:
        print(tag)
else:
    print("'tags' column does not exist in the DataFrame.")

Unique tags:
large_porch
community_swimming_pool
garage_3_or_more
waterfront
golf_course_lot_or_frontage
golf_course_view
kitchen_island
clubhouse
private_backyard
wrap_around_porch
washer_dryer
dining_room
beautiful_backyard
central_air
rental_property
well_water
community_park
community_golf
view
granite_kitchen
central_heat
water_view
recreation_facilities
community_horse_facilities
courtyard_entry
greenbelt
indoor_basketball_court
big_lot
pond
investment_opportunity
golf_course
smart_homes
security
community_gym
views
river_view
wine_cellar
family_room
ensuite
gated_community
rv_or_boat_parking
single_story
library
private_bathroom
no_hoa
cathedral_ceiling
carport
community_elevator
den_or_office
tennis_court
lake_view
swimming_pool
coffer_ceiling
modern_kitchen
community_spa_or_hot_tub
trails
outdoor_kitchen
community_boat_facilities
two_kitchen
community_outdoor_space
exposed_brick
pets_allowed
playground
master_suite
community_center
big_bathroom
mountain_view
horse_property
cit

In [100]:
# Data types of each column

data_types_list = combined_data.dtypes.tolist()

print("Data types of each column:")
print(data_types_list)

# There are objects, float64's

Data types of each column:
[dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64'), dtype('float64'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('O'), dtype('float64')]


In [101]:
# drop sales that dont include sales price

combined_data = combined_data.dropna(subset=['list_price'])

# Print the shape of the DataFrame to verify the number of rows after dropping

print("Shape of DataFrame after dropping rows with missing sales price:", combined_data.shape)

Shape of DataFrame after dropping rows with missing sales price: (7721, 67)


In [ ]:
# Drop sales that dont have a property type

# Display the property type column

print(combined_data['description.sub_type'])

# Drop N/A's

combined_data = combined_data.dropna(subset=['description.sub_type'])

# Print the shape of the DataFrame to verify the number of rows after dropping

print("Shape of DataFrame after dropping rows with missing property type:", combined_data.shape)


0       None
1       None
2       None
3       None
4       None
        ... 
8186     NaN
8187     NaN
8188     NaN
8189     NaN
8190     NaN
Name: description.sub_type, Length: 8191, dtype: object
Shape of DataFrame after dropping rows with missing property type: (1427, 67)


Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables here

encoded_df = pd.get_dummies(combined_data, columns=['location.address.city', 'location.address.state'])

# Display the encoded DataFrame
print(encoded_df)

# tags will have to be done manually

# Let's convert the 'tags' column to a set of unique tags

unique_tags = set(tag for tags_list in encoded_df['tags'] if tags_list is not None for tag in tags_list)

# Create a DataFrame to store the one-hot encoded tags

one_hot_tags_df = pd.DataFrame(0, index=encoded_df.index, columns=list(unique_tags))

# Iterate over each row and set the corresponding tag values to 1

for i, tags_list in enumerate(encoded_df['tags']):
    if tags_list is not None:
        for tag in tags_list:
            one_hot_tags_df.at[i, tag] = 1

# Concatenate the one-hot encoded tags DataFrame with the original DataFrame

encoded_df = pd.concat([encoded_df, one_hot_tags_df], axis=1)

# Drop the original 'tags' column 
encoded_df.drop('tags', axis=1, inplace=True)

# Display the resulting DataFrame

encoded_df.head()

          last_update_date                                               tags  \
5     2023-08-04T22:49:14Z  [carport, hill_or_mountain_view, ocean_view, s...   
12    2023-08-04T22:49:14Z  [carport, hill_or_mountain_view, ocean_view, s...   
18    2023-08-04T22:49:14Z  [carport, hill_or_mountain_view, ocean_view, s...   
23    2023-08-04T22:49:14Z  [carport, hill_or_mountain_view, ocean_view, s...   
27    2023-08-04T22:49:14Z  [carport, hill_or_mountain_view, ocean_view, s...   
...                    ...                                                ...   
7971  2024-01-03T19:35:24Z  [central_air, community_elevator, community_ou...   
7972  2024-01-03T19:41:48Z  [community_outdoor_space, disability_features,...   
7976  2024-01-03T02:46:30Z  [central_air, community_elevator, community_gy...   
7977  2024-01-10T18:23:43Z  [central_air, community_clubhouse, community_o...   
7978  2023-12-29T18:36:39Z  [central_air, community_elevator, community_gy...   

                           

,last_update_date,permalink,status,list_date,open_houses,branding,list_price,property_id,photos,community,...,master_bathroom,community_tennis_court,front_porch,garage_1_or_more,low_hoa,senior_community,white_kitchen,guest_parking,large_kitchen,elevator
5,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
12,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,None,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
18,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,None,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
23,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,[{'name': 'Platinum Keller Williams Realty Ala...,415000.0,7412309719,None,None,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Define a function to flatten the 'branding' column

def flatten_branding(row):
    branding_info = []
    try:
        for item in row:
            if isinstance(item, dict):
                branding_info.append(item['name'])
            elif isinstance(item, list):
                for sub_item in item:
                    if isinstance(sub_item, dict):
                        branding_info.append(sub_item['name'])
            elif pd.isna(item):  
                branding_info.append(None)
            elif isinstance(item, float):  
                pass
    except TypeError:  
        pass
    return branding_info

# Apply the flatten_branding function to each row to get a list of all brand names

encoded_df['branding_info'] = encoded_df['branding'].apply(flatten_branding)

# Drop the original 'branding' column

encoded_df.drop(columns=['branding'], inplace=True)

# Flatten the list of branding_info and extract unique brand names

unique_brands = set()
for branding_info in encoded_df['branding_info']:
    if branding_info is not None:
        unique_brands.update(branding_info)

# Perform one-hot encoding for unique brand names

for brand in unique_brands:
    encoded_df[brand] = encoded_df['branding_info'].apply(lambda x: 1 if x is not None and brand in x else 0)

# Drop the intermediate 'branding_info' column
encoded_df.drop(columns=['branding_info'], inplace=True)

# Display the resulting DataFrame

encoded_df.head()

C:\Users\turab\AppData\Local\Temp\ipykernel_16660\1972427617.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df[brand] = encoded_df['branding_info'].apply(lambda x: 1 if x is not None and brand in x else 0)
C:\Users\turab\AppData\Local\Temp\ipykernel_16660\1972427617.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df[brand] = encoded_df['branding_info'].apply(lambda x: 1 if x is not None and brand in x else 0)
C:\Users\turab\AppData\Local\Temp\ipykernel_16660\1972427617.py:39: PerformanceWarning: DataFr

,last_update_date,permalink,status,list_date,open_houses,list_price,property_id,photos,community,virtual_tours,...,Iron Valley Real Estate of Central PA - Carlisle Office,Iron Valley Real Estate Of Central Pa,Bhhs Utah Properties - Sv,Summit Sotheby's International Realty,Better Homes and Gardens Real Estate Metro Brokers,BancWise Realty,"Terry Howe & Associates, Inc",Howard Hanna,Berkshire Hathaway HomeServices New England Properties - Farmington,"RSR REALTORS, LLC"
5,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,None,None,...,0,0,0,0,0,0,0,0,0,0
12,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,None,None,...,0,0,0,0,0,0,0,0,0,0
18,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,None,None,...,0,0,0,0,0,0,0,0,0,0
23,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,None,None,...,0,0,0,0,0,0,0,0,0,0
27,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Drop irellevant columns

# Drop the 'photos' column and 'other_listings.rdc' column

encoded_df.drop(columns=['photos', 'virtual_tours'], inplace=True)

encoded_df.head()

,last_update_date,permalink,status,list_date,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,...,Iron Valley Real Estate of Central PA - Carlisle Office,Iron Valley Real Estate Of Central Pa,Bhhs Utah Properties - Sv,Summit Sotheby's International Realty,Better Homes and Gardens Real Estate Metro Brokers,BancWise Realty,"Terry Howe & Associates, Inc",Howard Hanna,Berkshire Hathaway HomeServices New England Properties - Farmington,"RSR REALTORS, LLC"
5,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,NaN,...,0,0,0,0,0,0,0,0,0,0
12,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,NaN,...,0,0,0,0,0,0,0,0,0,0
18,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,NaN,...,0,0,0,0,0,0,0,0,0,0
23,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,NaN,...,0,0,0,0,0,0,0,0,0,0
27,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Replace NaN values with 'None' for all 

encoded_df.fillna('None', inplace=True)

encoded_df.head()


C:\Users\turab\AppData\Local\Temp\ipykernel_16660\403460919.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  encoded_df.fillna('None', inplace=True)


,last_update_date,permalink,status,list_date,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,...,Iron Valley Real Estate of Central PA - Carlisle Office,Iron Valley Real Estate Of Central Pa,Bhhs Utah Properties - Sv,Summit Sotheby's International Realty,Better Homes and Gardens Real Estate Metro Brokers,BancWise Realty,"Terry Howe & Associates, Inc",Howard Hanna,Berkshire Hathaway HomeServices New England Properties - Farmington,"RSR REALTORS, LLC"
5,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,...,0,0,0,0,0,0,0,0,0,0
12,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,...,0,0,0,0,0,0,0,0,0,0
18,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,...,0,0,0,0,0,0,0,0,0,0
23,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,...,0,0,0,0,0,0,0,0,0,0
27,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,sold,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# One-hot encode the 'status' 

encoded_df['status_sold'] = (encoded_df['status'] == 'sold').astype(int)
encoded_df['status_for_sale'] = (encoded_df['status'] == 'for sale').astype(int)

# Drop the original 'status' column

encoded_df.drop(columns=['status'], inplace=True)

encoded_df.head()


C:\Users\turab\AppData\Local\Temp\ipykernel_16660\1131935107.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df['status_sold'] = (encoded_df['status'] == 'sold').astype(int)
C:\Users\turab\AppData\Local\Temp\ipykernel_16660\1131935107.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df['status_for_sale'] = (encoded_df['status'] == 'for sale').astype(int)


,last_update_date,permalink,list_date,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,matterport,...,Bhhs Utah Properties - Sv,Summit Sotheby's International Realty,Better Homes and Gardens Real Estate Metro Brokers,BancWise Realty,"Terry Howe & Associates, Inc",Howard Hanna,Berkshire Hathaway HomeServices New England Properties - Farmington,"RSR REALTORS, LLC",status_sold,status_for_sale
5,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,...,0,0,0,0,0,0,0,0,1,0
12,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,...,0,0,0,0,0,0,0,0,1,0
18,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,...,0,0,0,0,0,0,0,0,1,0
23,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,...,0,0,0,0,0,0,0,0,1,0
27,2023-08-04T22:49:14Z,11909-Glacier-Hwy-Apt-105_Juneau_AK_99801_M741...,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# Split the 'permalink' column into separate columns
encoded_df[['address', 'city', 'state', 'zip_code', 'unique_identifier']] = encoded_df['permalink'].str.split('_', expand=True)

# Drop the unnecessary columns
encoded_df.drop(columns=['permalink', 'unique_identifier'], inplace=True)

encoded_df.head()


C:\Users\turab\AppData\Local\Temp\ipykernel_16660\3659094003.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df[['address', 'city', 'state', 'zip_code', 'unique_identifier']] = encoded_df['permalink'].str.split('_', expand=True)
C:\Users\turab\AppData\Local\Temp\ipykernel_16660\3659094003.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  encoded_df[['address', 'city', 'state', 'zip_code', 'unique_identifier']] = encoded_df['permalink'].str.split('_', expand=True)
C:\Users\turab\AppData\Local\Temp\ipykernel_16660\36

,last_update_date,list_date,open_houses,list_price,property_id,community,listing_id,price_reduced_amount,matterport,primary_photo.href,...,"Terry Howe & Associates, Inc",Howard Hanna,Berkshire Hathaway HomeServices New England Properties - Farmington,"RSR REALTORS, LLC",status_sold,status_for_sale,address,city,state,zip_code
5,2023-08-04T22:49:14Z,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,None,...,0,0,0,0,1,0,11909-Glacier-Hwy-Apt-105,Juneau,AK,99801
12,2023-08-04T22:49:14Z,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,None,...,0,0,0,0,1,0,11909-Glacier-Hwy-Apt-105,Juneau,AK,99801
18,2023-08-04T22:49:14Z,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,None,...,0,0,0,0,1,0,11909-Glacier-Hwy-Apt-105,Juneau,AK,99801
23,2023-08-04T22:49:14Z,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,None,...,0,0,0,0,1,0,11909-Glacier-Hwy-Apt-105,Juneau,AK,99801
27,2023-08-04T22:49:14Z,2023-08-21T21:01:22.000000Z,None,415000.0,7412309719,None,2958925334,None,False,None,...,0,0,0,0,1,0,11909-Glacier-Hwy-Apt-105,Juneau,AK,99801


In [ ]:
# Import LabelEncoder from scikit-learn

from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder

label_encoder = LabelEncoder()

# Iterate over each column

for column in encoded_df.columns:
    # Check if the column is not numeric

    if encoded_df[column].dtype != 'int' and encoded_df[column].dtype != 'float':
        # Convert non-numeric values to string

        encoded_df[column] = encoded_df[column].astype(str)
        # Encode the column

        encoded_df[column] = label_encoder.fit_transform(encoded_df[column])

# Check the encoded dataframe

print(encoded_df.head())


    last_update_date  list_date  open_houses  list_price  property_id  \
5                  1         52            0         137          223   
12                 1         52            0         137          223   
18                 1         52            0         137          223   
23                 1         52            0         137          223   
27                 1         52            0         137          223   

    community  listing_id  price_reduced_amount  matterport  \
5           0          56                    51           0   
12          0          56                    51           0   
18          0          56                    51           0   
23          0          56                    51           0   
27          0          56                    51           0   

    primary_photo.href  ...  Terry Howe & Associates, Inc  Howard Hanna  \
5                    0  ...                             0             0   
12                   0  ...     

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE such as using central tendency?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- If you replace cities or states with numerical values, make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Drop columns that aren't needed.
- Don't keep the list price because it will be too close to the sale price.

In [ ]:
from sklearn.model_selection import train_test_split

X = encoded_df.drop('list_price', axis=1)
y = encoded_df['list_price']

# Perform train-test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the train and test sets

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (2096, 524)
X_test shape: (524, 524)
y_train shape: (2096,)
y_test shape: (524,)


In [86]:
# Save data to CSV files

X_train.to_csv('x_train.csv', index=False)
X_test.to_csv('x_test.csv', index=False)
y_train_df.to_csv('y_train.csv', index=False)
y_test_df.to_csv('y_test.csv', index=False)


**STRETCH**

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized